In [1]:
import os
import sys
import jieba

data_dir = "./zh_dataset/"
train_filepath = os.path.join(data_dir, "cnews.train.txt")
val_filepath = os.path.join(data_dir, "cnews.val.txt")
test_filepath = os.path.join(data_dir, "cnews.test.txt")

seg_train_filepath = os.path.join(data_dir, "cnews.train.seg.txt")
seg_val_filepath = os.path.join(data_dir, "cnews.val.seg.txt")
seg_test_filepath = os.path.join(data_dir, "cnews.test.seg.txt")

vocab_filepath = os.path.join(data_dir, "vocab.txt")
category_filepath = os.path.join(data_dir, "category.txt")

In [2]:
def generate_seg(input_file, output_seg_file):
    with open(input_file, "r") as f:
        lines = f.readlines()
    
    with open(output_seg_file, "w") as f:
        for line in lines:
            if line.strip() == "":
                continue

            label, content = line.strip().split("\t")
            label = label.strip()
            content = content.strip()

            word_iter = jieba.cut(content)
            content_list = []

            for word in word_iter:
                word = word.strip()
                if word != "":
                    content_list.append(word)
            
            f.write("%s\t%s\n"%(label, " ".join(content_list)))
        
generate_seg(train_filepath, seg_train_filepath)
generate_seg(val_filepath, seg_val_filepath)
generate_seg(test_filepath, seg_test_filepath)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.583 seconds.
Prefix dict has been built successfully.


In [3]:
def generate_vocab(seg_file, vocab_file):
    with open(seg_file, "r") as f:
        lines = f.readlines()
    
    vocab_dict = {}
    with open(vocab_file, "w") as f:
        for line in lines:
            if line.strip() == "":
                continue
            label, content = line.strip().split("\t")
            word_iter = content.strip().split(" ")
            for word in word_iter:
                vocab_dict.setdefault(word, 0)
                vocab_dict[word] += 1
        
        sorted_vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
        f.write("<UNK>\t10000000\n")
        for word, count in sorted_vocab:
            f.write("%s\t%s\n"%(word, count))

generate_vocab(seg_train_filepath, vocab_filepath)

In [4]:
def generate_category(input_file, category_file):
    with open(input_file, "r") as f:
        lines = f.readlines()
    
    category_dict = {}
    with open(category_file, "w") as f:
        for line in lines:
            if line.strip() == "":
                continue
            label, content = line.strip().split("\t")
            category_dict.setdefault(label, 0)
            category_dict[label] += 1
            
        sorted_category = sorted(category_dict.items(), key=lambda x:x[1], reverse=True)
        for label, count in sorted_category:
            f.write("%s\t%s\n"%(label, count))
            
generate_category(train_filepath, category_filepath)